Predict whether a voltage rise will occur or not in the next period . 

See [VoltageRiseBinTrain](voltageRiseBin_Train.ipynb) for the training of the RNN

---

#### Import modules to be used

In [1]:
#Import Modules 
import pandas as pd
import pandapower as pp
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm # Profiling 
import seaborn as sbn
import pickle, sys, importlib,  time
import os
from pickle import load
import tensorflow as tf
import joblib

#### Import my own modules

In [2]:
# import psutil
function_folder = 'py_files/' 
# Add function folder to path if it is not already
if function_folder not in sys.path: sys.path.append(function_folder)

import myFunctions as mf
from myFunctions import network_folder, excel_folder, py_folder, Δt, attr_list

In [3]:
fut_known = joblib.load('pickle_files/simulationResults/KnownFuture.pkl')


#### Import and dataClead files for component of the network

In [4]:
#Import Networks
net_civaux=pp.from_pickle(f'{network_folder}CIVAUX.p')
net_stlaurent=pp.from_pickle(f'{network_folder}ST LAURENT.p')

In [5]:
#Load files
file_p_inj_0013 = 'Prod_HTA/PROD_Bis/P0013/PROD-BATISOLAIRE 6-CIVAUX - Actif injecte (P-).csv'
file_p_inj_0018 = 'Prod_HTA/PROD_Bis/P0018/PROD-SUN POITOU 2516 (Z.I de la Pitage)-LHOMMAIZE - Actif injecte (P-).csv'


# The  commissioning of the Prod P0100 is recent (2022). I therefore use the data of the closer energy 
# producer that is P0058  and consider it as that of Prod P0100 
file_p_inj_0100 = 'Prod_HTA/PROD_Bis/PROD-SERGIES 2204 (LA ROCHE A CORNUCHON)-PINDRAY - Actif P-.csv'

file_prod_bt_total = 'PROD_BTSUP36_SAINT LAURENT.csv'
file_cons_total = 'CONSO_POSTE_SAINT LAURENT.csv'


# Get files data 
p_mw_0013 = mf.readAndReshape_input(file_p_inj_0013 ,excel_folder, )
p_mw_0018 = mf.readAndReshape_input(file_p_inj_0018 ,excel_folder,)
p_mw_0100 = mf.readAndReshape_input(file_p_inj_0100 ,excel_folder,)


p_mw_prod_bt_total = mf.readAndReshape_input(file_prod_bt_total, excel_folder)
p_mw_cons_total = mf.readAndReshape_input(file_cons_total, excel_folder)

# Create dict for all HT producers
dict_prod_hv = {'P0013': p_mw_0013[:len(p_mw_0100)], 
                'P0018': p_mw_0018[:len(p_mw_0100)],
                'P0100': p_mw_0100[:len(p_mw_0100)]
               }
# Create index to use for dataframe
per_index = pd.period_range('01 01 2020', periods=len(p_mw_0100), freq='10T')

# Use the create dict to create a dataFrame for Prod P0100
df_prodP0100 = pd.DataFrame(p_mw_0100, index=per_index)

# Use the create dict to create a dataFrame for all HT producers
df_prodHT = pd.DataFrame(dict_prod_hv, index=per_index)

# Dataframe prod BT 
per_index = pd.period_range('01 01 2020', periods=len(p_mw_prod_bt_total), freq='10T')
df_prod_bt_total = pd.DataFrame(p_mw_prod_bt_total, index=per_index, columns=['Prod_BT'])


# Dataframe Conso BT 
per_index = pd.period_range('01 01 2020', periods=len(p_mw_cons_total), freq='10T')
df_cons_total = pd.DataFrame(p_mw_cons_total, index=per_index, columns=['Cons'])
# Data cleaning on Consumption
previous_days = df_cons_total[(per_index>='2022 02 12') & (per_index<='2022 02 21 23:50')]
following_days = df_cons_total[(per_index>='2022 03 03') & (per_index<='2022 03 12 23:50')]
# # Put the interpolated data into the dataframe
df_cons_total[(per_index>='2022 02 21') & (per_index<='2022 03 02 23:50')] = (np.array(following_days) + 
                                                                              np.array(previous_days) )/2


# Get total Power of BT producers
# Bt producers are indexed by the name None
max_p_mw_total_prodBT = net_civaux.sgen.max_p_mw[net_civaux.sgen.name.isna()].sum()

# # Get total power of load in the network
max_p_mw_total_load = net_civaux.load.max_p_mw.sum()

# Select relevant data up to 2022 06 01
df_prodHT = df_prodHT[df_prodHT.index<='2022 06 01 23:50']
df_prod_bt_total = df_prod_bt_total[df_prod_bt_total.index<='2022 06 01 23:50']
df_cons_total = df_cons_total[df_cons_total.index<='2022 06 01 23:50']


# Extract only dailight period i.e. from 07am to 7PM
# The daylight period is considered to be defined betwenn 07am and 7Pm excluded. 
h_start_end = ('07:00','18:50') # for the persistance model, the previous period i.e. 06:50 
                                # is needed to compute the first instant i.e. 07:00
per_index = df_prodHT.index
per_daylight = ( pd.Series(index=per_index.to_timestamp(), dtype=object).between_time(*h_start_end) ).index.to_period('10T')
day_tot_per = len(per_daylight[(per_daylight.year==2020)&(per_daylight.month==1)&(per_daylight.day==1)])


# Put all the data in a unique dataframe
df_data = pd.concat([df_cons_total, df_prod_bt_total, df_prodHT], axis=1).loc[per_daylight]

# # Extract only the relavant testing set since the training set covers the first part of the data
df_final = df_data[df_data.index>='2021 06 01']
per_index = df_final.index
per_index2 = ( pd.Series(index=per_index.to_timestamp(), dtype=object
                           ).between_time('07:00','18:50') ).index.to_period('10T')

# # Separate training and testing set 
df_test = df_data[df_data.index>='2021 06 01']

In [7]:
vm_mu_max, vm_mu_min = 1.0250, 0.95  # Choosen 


In [8]:
# Create a vectorize version of int64
vector_int = np.vectorize(np.int64)

# use the vector int to convert the voltage rise in a binary variable 
var_bin = vector_int(fut_known['voltage_rise_df'].No_Control > vm_mu_max)

var_bin_df = pd.DataFrame(data=var_bin, index=fut_known['voltage_rise_df'].index, columns=['vRise_bin'])

df_test_bin = pd.concat([df_test, var_bin_df], axis=1)

### Import RNN

In [9]:
import joblib

In [10]:
# Load scaler and RNN from file
scaler = joblib.load('pickle_files/RNN/StLaurent_bin_vRise_scaler.plk')
lstm_model = tf.keras.models.load_model('pickle_files/RNN/StLaurent_bin_vRise_model')

#### Define history

In [12]:
histTot = df_test_bin.rolling(lstm_model.input_shape[1]) # Create a rolling windows to get the history

#### Predict values

In [15]:
tqdm

tqdm.std.tqdm

In [ ]:
hist_list = list(histTot)
len_hist = len(hist_list)
n0  = lstm_model.input_shape[1]                 # The first elem 
pred_per, pred = [],[]             
                
for hist_index in tqdm(range(n0, len_hist)):
    cur_hist = hist_list[hist_index]          # current hystory
    
    # run prediction for each period 
    pred_var, pred_per_var = mf.predictionBin_bloc(rnn_model=lstm_model, 
                                                   fitting_scaler=scaler, 
                                                   history= cur_hist, sig_thresh=0.5 )
    
    pred_per.append(pred_per_var)
    pred.append(pred_var)
    


In [98]:
predicted_values = pd.DataFrame(data = np.array(pred),
                                index=pred_per, 
                                columns = ['V_rise_Pred'] )

per_index3 = ( pd.Series(index=predicted_values.index.to_timestamp(), dtype=object
                           ).between_time('08:10','18:50') ).index.to_period('10T')

In [99]:
from sklearn.metrics import classification_report, confusion_matrix

In [100]:
print(classification_report(var_bin_df.loc[per_index3],predicted_values.loc[per_index3]))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98     21242
           1       0.78      0.89      0.83      2548

    accuracy                           0.96     23790
   macro avg       0.88      0.93      0.90     23790
weighted avg       0.96      0.96      0.96     23790



In [101]:
print(confusion_matrix(var_bin_df.loc[per_index3],predicted_values.loc[per_index3]))

[[20612   630]
 [  293  2255]]


In [102]:
joblib.dump(predicted_values.loc[per_index3],'pickle_files/simulationResults/Binary_Voltage_Rise_Predicted.pkl')

['pickle_files/simulationResults/Binary_Voltage_Rise_Predicted.pkl']